### Stanford CS 20SI: Tensorflow for Deep Learning Research
* This notebook contains the exercise from [CS20SI](https://web.stanford.edu/class/cs20si/)
* This notebook is used for purely exercise purpose
* All credit belongs to CS20SI team

In [1]:
import numpy as np
import tensorflow as tf

### simple tf

In [2]:
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)

with tf.Session() as sess:
    print(sess.run(x))

5


In [3]:
a = tf.constant([[2, 2]], name='a')
b = tf.constant([[0, 1], [2, 2]], name='b')
x = tf.multiply(a, b, name='dot_product')
y = tf.matmul(a, b, name='cross_product')

with tf.Session() as sess:
    print(sess.run(x), '\n')
    print(sess.run(y))

[[0 2]
 [4 4]] 

[[4 6]]


In [4]:
x = tf.zeros([2, 3], tf.int32) 
y = tf.zeros_like(x, optimize=True)
with tf.Session() as sess:
    print(sess.run(x), '\n')
    print(sess.run(y))

[[0 0 0]
 [0 0 0]] 

[[0 0 0]
 [0 0 0]]


In [5]:
with tf.Session() as sess:
    print(sess.run(tf.linspace(10.0, 13.0, 4)))
    print(sess.run(tf.range(5)))

[ 10.  11.  12.  13.]
[0 1 2 3 4]


In [6]:
samples = tf.multinomial(tf.constant([[1., 1., 1]]), 5)
with tf.Session() as sess:
    print(sess.run(samples))

[[0 0 2 0 1]]


In [7]:
t_2 = [[True, False, False],
       [False, False, True],
       [False, True, False]] 
x = tf.zeros_like(t_2)  # ==> 2x2 tensor, all elements are False
y = tf.ones_like(t_2)  # ==> 2x2 tensor, all elements are True
with tf.Session() as sess:
    print(sess.run([x, y]))

[array([[False, False, False],
       [False, False, False],
       [False, False, False]], dtype=bool), array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]], dtype=bool)]


In [8]:
tf.reset_default_graph()
with tf.Session() as sess:
    with tf.variable_scope('tmp1', reuse=None) as scope:
        a = tf.get_variable('a', [3], dtype=tf.float32)
        b = tf.get_variable('b', [3], dtype=tf.float32)
        
    sess.run(tf.global_variables_initializer())
    print(a.name, '\t', sess.run(a))
    print(b.name, '\t', sess.run(b))
    
    with tf.variable_scope('tmp1', reuse=None) as scope:
        c = a + 1
        d = b + 1
        
    print(c.name, '\t', sess.run(a), '\t', sess.run(c))
    print(c.name, '\t', sess.run(b), '\t', sess.run(d))

tmp1/a:0 	 [-0.54506922 -0.52918458  0.28080463]
tmp1/b:0 	 [ 0.75396538 -0.01538348  0.01470685]
tmp1_1/add:0 	 [-0.54506922 -0.52918458  0.28080463] 	 [ 0.45493078  0.47081542  1.28080463]
tmp1_1/add:0 	 [ 0.75396538 -0.01538348  0.01470685] 	 [ 1.75396538  0.98461652  1.01470685]


In [9]:
x = tf.Variable(2.0)
y = 2.0 * (x ** 3)
z = 3.0 + y ** 2
grad_z = tf.gradients(z, [x, y])
with tf.Session() as sess:
    sess.run(x.initializer)
    print(sess.run(grad_z))

[768.0, 32.0]


### feed_dict

In [10]:
a = tf.placeholder(dtype=tf.float32, shape=[3])
b = tf.constant(value=[5, 5, 5], dtype=tf.float32)
c = a + b

with tf.Session() as sess:
    print(sess.run(c, feed_dict={a: [1, 2, 3]}))

[ 6.  7.  8.]


In [11]:
a = tf.add(2, 5)
b = tf.multiply(a, 3)

with tf.Session() as sess:
    print(sess.run([a, b]))
    
    replace_dict = {a: 15}
    print(sess.run([a, b], feed_dict=replace_dict))

[7, 21]
[array(15, dtype=int32), 45]


### variables

In [12]:
W = tf.Variable(10)
assign_op = W.assign(100)

with tf.Session() as sess:
    sess.run(W.initializer)
    print(W.eval()) # >> 10
    print(sess.run(assign_op)) # >> 100
    print(W.eval()) # >> 100

10
100
100


In [13]:
W = tf.Variable(10)
sess1 = tf.Session()
sess2 = tf.Session()

# You have to initialize W at each session
sess1.run(W.initializer)
sess2.run(W.initializer)

print(sess1.run(W.assign_add(10))) # >> 20
print(sess2.run(W.assign_sub(2))) # >> 8

print(sess1.run(W.assign_add(100))) # >> 120
print(sess2.run(W.assign_sub(50))) # >> -42

sess1.close()
sess2.close()

20
8
120
-42


In [26]:
a = np.array([[1, 2, 3], 
              [4, 5, 6], 
              [7, 8, 9]])
b = np.array([[2, 4, 6], 
              [5, 7, 9], 
              [8, 10, 12]])

x = tf.placeholder(tf.float32, [None, 3])
y = tf.placeholder(tf.float32, [None, 3])

n = len(a)

z1 = tf.pow(y - x, 2)
z2 = tf.reduce_sum(z1, axis=1)

loss1 = tf.reduce_mean(tf.pow(y - x, 2))
loss2 = tf.reduce_mean(tf.reduce_sum(tf.pow(y - x, 2), axis=1))
loss3 = tf.squared_difference(y, x)
loss4 = tf.losses.mean_squared_error(y, x)
loss5 = tf.reduce_sum(z1) / n

with tf.Session() as sess:
    print(sess.run(z1, feed_dict={x: a, y: b}))
    print(sess.run(z2, feed_dict={x: a, y: b}))
    print(sess.run(loss1, feed_dict={x: a, y: b}))
    print(sess.run(loss2, feed_dict={x: a, y: b}))
    print(sess.run(loss3, feed_dict={x: a, y: b}))
    print(sess.run(loss4, feed_dict={x: a, y: b}))
    print(sess.run(loss5, feed_dict={x: a, y: b}))

[[ 1.  4.  9.]
 [ 1.  4.  9.]
 [ 1.  4.  9.]]
[ 14.  14.  14.]
4.66667
14.0
[[ 1.  4.  9.]
 [ 1.  4.  9.]
 [ 1.  4.  9.]]
4.66667
14.0
